In [43]:
import numpy as np
import pandas as pd 
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import nltk 
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import requests
from datetime import datetime

In [48]:
sid_obj = SentimentIntensityAnalyzer()

# company news
summary is not full

should be grouped by date and mean sentiment

In [49]:
r = requests.get('https://finnhub.io/api/v1/company-news?symbol=ZM&from=2021-03-01&to=2021-03-09&token=c2si65iad3ic1qis06lg')
print(r.json()[0]['summary'])
news_df = pd.DataFrame(r.json())
news_df['datetime'] = [datetime.utcfromtimestamp(i).strftime('%Y-%m-%d') for i in news_df.datetime]
news_df.drop(['id','image', 'related','source', 'url'], axis=1, inplace=True)
news_df['headline_sentiment'] = [sid_obj.polarity_scores(c)['compound'] for c in news_df['headline']]
news_df['summary_sentiment'] = [sid_obj.polarity_scores(c)['compound'] for c in news_df['summary']]
news_df.head()

The stock market has experienced some extreme volatility in recent weeks, but JJ Kinahan, TD Ameritrade's chief market strategist, says retail traders have still been buying...


,category,datetime,headline,summary,headline_sentiment,summary_sentiment
0,company,2021-03-10,"Schwab, TD Ameritrade Experts Talk Retail Trad...",The stock market has experienced some extreme ...,0.2500,0.5573
1,company,2021-03-09,Zoom Video Communications Inc. stock outperfor...,Shares of Zoom Video Communications Inc. climb...,0.5106,0.6486
2,company,2021-03-09,Why Comcast Could Be Ready To Cash In On Hulu ...,The streaming market has seen explosive growth...,0.3612,0.3818
3,company,2021-03-09,IPO: Reducing The Fear Of Missing Out,Renaissance IPO ETF provides a chance to inves...,-0.6597,0.1531
4,company,2021-03-09,ARK Innovation And The Demise Of Growth Investing,The ARK Innovation ETF has erased all of its 2...,0.6369,0.3612


In [50]:
news_df[news_df.datetime == '2021-03-09']

,category,datetime,headline,summary,headline_sentiment,summary_sentiment
1,company,2021-03-09,Zoom Video Communications Inc. stock outperfor...,Shares of Zoom Video Communications Inc. climb...,0.5106,0.6486
2,company,2021-03-09,Why Comcast Could Be Ready To Cash In On Hulu ...,The streaming market has seen explosive growth...,0.3612,0.3818
3,company,2021-03-09,IPO: Reducing The Fear Of Missing Out,Renaissance IPO ETF provides a chance to inves...,-0.6597,0.1531
4,company,2021-03-09,ARK Innovation And The Demise Of Growth Investing,The ARK Innovation ETF has erased all of its 2...,0.6369,0.3612
5,company,2021-03-09,Covid Conundrum: Zoom In The Spotlight Once Mo...,The end may at last be coming for the pandemic...,0.0000,0.5859
6,company,2021-03-09,Zoom Founder Eric Yuan Transfers $6 Billion of...,The videoconferencing platform’s chief executi...,0.4215,0.0000
7,company,2021-03-09,Wall Street Breakfast: Correcting The Correction?,Listen on the go! A daily podcast of Wall Stre...,0.0000,0.0000


In [55]:
news_dates = news_df.groupby(['datetime']).mean()
news_dates #can be used for merging on date

,headline_sentiment,summary_sentiment
datetime,,
2021-03-01,0.123812,0.260354
2021-03-02,-0.048075,0.163240
2021-03-03,0.064543,0.394900
2021-03-04,-0.050960,0.137180
2021-03-05,0.092900,0.229513
2021-03-06,0.670500,0.840200
2021-03-07,0.226300,0.557400
2021-03-08,0.123670,-0.063940
2021-03-09,0.181500,0.304371


# News Sentiment
Get company's news sentiment and statistics. This endpoint is only available for US companies.

get only one raw for the last week

buzz
Statistics of company news in the past week.

articlesInLastWeek
buzz
weeklyAverage
companyNewsScore
News score.

sectorAverageBullishPercent
Sector average bullish percent.

sectorAverageNewsScore
Sectore average score.

sentiment
News sentiment.

bearishPercent
bullishPercent
symbol
Requested symbol.

In [11]:
r = requests.get('https://finnhub.io/api/v1/news-sentiment?symbol=ZM&token=c2si65iad3ic1qis06lg')
print(r.json())

{'buzz': {'articlesInLastWeek': 50, 'buzz': 0.9708, 'weeklyAverage': 51.5}, 'companyNewsScore': 0.6739, 'sectorAverageBullishPercent': 0.6643, 'sectorAverageNewsScore': 0.5393, 'sentiment': {'bearishPercent': 0, 'bullishPercent': 1}, 'symbol': 'ZM'}


# Basic Financials
Get company basic financials such as margin, P/E ratio, 52-week high/low etc.

quarterly /  annually

In [20]:
r = requests.get('https://finnhub.io/api/v1/stock/metric?symbol=ZM&metric=all&token=c2si65iad3ic1qis06lg')
print(r.json()['metric'])
print(r.json()['series']['annual'])

{'10DayAverageTradingVolume': 4.40122, '13WeekPriceReturnDaily': 5.77673, '26WeekPriceReturnDaily': -7.71013, '3MonthAverageTradingVolume': 69.97736, '52WeekHigh': 588.84, '52WeekHighDate': '2020-10-19', '52WeekLow': 216.75, '52WeekLowDate': '2020-06-12', '52WeekPriceReturnDaily': 64.99302, '5DayPriceReturnDaily': 9.04762, 'assetTurnoverAnnual': 0.80493, 'assetTurnoverTTM': 0.82436, 'beta': None, 'bookValuePerShareAnnual': 13.15203, 'bookValuePerShareQuarterly': 14.23621, 'bookValueShareGrowth5Y': None, 'capitalSpendingGrowth5Y': None, 'cashFlowPerShareAnnual': 2.35192, 'cashFlowPerShareTTM': 3.01615, 'cashPerSharePerShareAnnual': 14.45997, 'cashPerSharePerShareQuarterly': 15.92322, 'currentDividendYieldTTM': 0, 'currentEv/freeCashFlowAnnual': 168.5417, 'currentEv/freeCashFlowTTM': 137.7837, 'currentRatioAnnual': 3.80396, 'currentRatioQuarterly': 3.49458, 'dividendGrowthRate5Y': None, 'dividendPerShare5Y': None, 'dividendPerShareAnnual': None, 'dividendYield5Y': None, 'dividendYieldInd

# Recommendation Trends
Get latest analyst recommendation trends for a company.

per month

In [38]:
r = requests.get('https://finnhub.io/api/v1/stock/recommendation?symbol=ZM&token=c2si65iad3ic1qis06lg')
print(r.json()[1])

recommendation_df = pd.DataFrame(r.json())
recommendation_df.head()

{'buy': 11, 'hold': 13, 'period': '2021-05-01', 'sell': 0, 'strongBuy': 10, 'strongSell': 1, 'symbol': 'ZM'}


,buy,hold,period,sell,strongBuy,strongSell,symbol
0,9,14,2021-06-01,0,10,1,ZM
1,11,13,2021-05-01,0,10,1,ZM
2,12,14,2021-04-01,1,9,1,ZM
3,10,16,2021-03-01,1,9,0,ZM
4,10,16,2021-02-01,1,9,0,ZM


# Social Sentiment
Get social sentiment for stocks on Reddit and Twitter. This endpoint is currently in Beta.

several per day

In [37]:
r = requests.get('https://finnhub.io/api/v1/stock/social-sentiment?symbol=ZM&token=c2si65iad3ic1qis06lg&from=2021-06-01')
print(r.json()['reddit'][0:10])
reddit_df = pd.DataFrame(r.json()['reddit'])
reddit_df.head()

[{'atTime': '2021-06-13 22:00:00', 'mention': 1, 'positiveScore': 0, 'negativeScore': 0.980246, 'positiveMention': 0, 'negativeMention': 1, 'score': -0.980246}, {'atTime': '2021-06-13 00:00:00', 'mention': 26, 'positiveScore': 0, 'negativeScore': 0, 'positiveMention': 0, 'negativeMention': 0, 'score': 0}, {'atTime': '2021-06-12 23:00:00', 'mention': 86, 'positiveScore': 0, 'negativeScore': 0, 'positiveMention': 0, 'negativeMention': 0, 'score': 0}, {'atTime': '2021-06-12 22:00:00', 'mention': 29, 'positiveScore': 0, 'negativeScore': 0, 'positiveMention': 0, 'negativeMention': 0, 'score': 0}, {'atTime': '2021-06-12 21:00:00', 'mention': 17, 'positiveScore': 0, 'negativeScore': 0, 'positiveMention': 0, 'negativeMention': 0, 'score': 0}, {'atTime': '2021-06-11 21:00:00', 'mention': 1, 'positiveScore': 0.9996642, 'negativeScore': 0, 'positiveMention': 1, 'negativeMention': 0, 'score': 0.9996642}, {'atTime': '2021-06-11 18:00:00', 'mention': 1, 'positiveScore': 0, 'negativeScore': 0.9149015

,atTime,mention,positiveScore,negativeScore,positiveMention,negativeMention,score
0,2021-06-13 22:00:00,1,0.0,0.980246,0,1,-0.980246
1,2021-06-13 00:00:00,26,0.0,0.000000,0,0,0.000000
2,2021-06-12 23:00:00,86,0.0,0.000000,0,0,0.000000
3,2021-06-12 22:00:00,29,0.0,0.000000,0,0,0.000000
4,2021-06-12 21:00:00,17,0.0,0.000000,0,0,0.000000


# list of companies

8124

description
Symbol description

displaySymbol
Display symbol name.

symbol
Unique symbol used to identify this symbol used in /stock/candle endpoint.

type
Security type.


In [35]:
r = requests.get('https://finnhub.io/api/v1/search?token=c2si65iad3ic1qis06lg')
print(r.json()['result'][0:10])
print(len(r.json()['result']))

[{'description': 'Forex.com AUD/JPY', 'displaySymbol': 'AUD/JPY', 'symbol': 'FOREX:401484303', 'type': 'FX'}, {'description': 'ZB QUN/BTC', 'displaySymbol': 'QUN/BTC', 'symbol': 'ZB:qun_btc', 'type': 'Crypto'}, {'description': 'Binance NMR/USDT', 'displaySymbol': 'NMR/USDT', 'symbol': 'BINANCE:NMRUSDT', 'type': 'Crypto'}, {'description': 'Kucoin LYM/USDT', 'displaySymbol': 'LYM/USDT', 'symbol': 'KUCOIN:LYM-USDT', 'type': 'Crypto'}, {'description': 'BITFINEX GOT/USD', 'displaySymbol': 'GOT/USD', 'symbol': 'BITFINEX:GOTUSD', 'type': 'Crypto'}, {'description': 'Huobi SALT/BTC', 'displaySymbol': 'SALT/BTC', 'symbol': 'HUOBI:saltbtc', 'type': 'Crypto'}, {'description': 'FXPIG UKOIL UKOIL', 'displaySymbol': 'UKOIL352/USD', 'symbol': 'FXPIG:352', 'type': 'Oil'}, {'description': 'BITFINEX ETP/ETH', 'displaySymbol': 'ETP/ETH', 'symbol': 'BITFINEX:ETPETH', 'type': 'Crypto'}, {'description': 'Binance TRX/PAX', 'displaySymbol': 'TRX/PAX', 'symbol': 'BINANCE:TRXPAX', 'type': 'Crypto'}, {'descriptio